In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [4]:
IMG_SIZE = 224
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255
)

def load_data(data_path):
    train_data = train_datagen.flow_from_directory(
        data_path,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )

    val_data = train_datagen.flow_from_directory(
        data_path,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )
    
    return train_data, val_data

train_orig, val_orig = load_data("C:/Users/PC/output_original_images")
train_proc, val_proc = load_data("C:/Users/PC/output_processed_images")


Found 640 images belonging to 4 classes.
Found 160 images belonging to 4 classes.
Found 5120 images belonging to 4 classes.
Found 1280 images belonging to 4 classes.


Bu Python kodu, görüntü veri setlerini yüklemek ve ön işlemek için Keras'ın `ImageDataGenerator` sınıfını kullanır. İlk olarak, `IMG_SIZE` değişkeni 224 olarak ve `BATCH_SIZE` değişkeni 32 olarak tanımlanır. `ImageDataGenerator` nesnesi, veri artırma ve ön işleme adımlarını yapılandırmak için oluşturulur. Bu örnekte, veri setinin %20'si doğrulama için ayrılır (`validation_split=0.2`) ve tüm görüntü pikselleri 0 ile 1 arasına ölçeklendirilir (`rescale=1./255`).

`load_data` adında bir fonksiyon tanımlanır. Bu fonksiyon, bir veri yolu (`data_path`) alır ve bu yoldaki görüntüleri eğitim ve doğrulama kümelerine ayırarak yükler. `flow_from_directory` metodu kullanılır. Bu metot, belirtilen dizindeki alt klasörleri (kategorileri) otomatik olarak tanır ve görüntüleri bu kategorilere göre etiketler. `target_size` parametresi, tüm görüntülerin belirtilen boyuta (224x224) yeniden boyutlandırılmasını sağlar. `batch_size`, her bir eğitim veya doğrulama adımında kaç görüntü işleneceğini belirler. `class_mode='categorical'` parametresi, sınıf etiketlerinin one-hot encoding formatında olmasını sağlar. `subset` parametresi, veri setinin hangi bölümünün (eğitim veya doğrulama) yükleneceğini belirtir. `shuffle=True` parametresi, eğitim verisinin karıştırılmasını sağlarken, doğrulama verisinin sırası korunur (`shuffle=False`).

Son olarak, `load_data` fonksiyonu iki kez çağrılır. İlk çağrı, "C:/Users/PC/output_original_images" yolundaki orijinal görüntüleri yükleyerek `train_orig` (eğitim verisi) ve `val_orig` (doğrulama verisi) değişkenlerine atar. İkinci çağrı ise "C:/Users/PC/output_processed_images" yolundaki işlenmiş (boyutlandırılmış ve potansiyel olarak başka işlemlerden geçmiş) görüntüleri yükleyerek `train_proc` (eğitim verisi) ve `val_proc` (doğrulama verisi) değişkenlerine atar. Bu şekilde, hem orijinal hem de işlenmiş görüntü veri setleri ayrı ayrı yüklenerek model eğitimi ve değerlendirmesi için hazır hale getirilir.

In [6]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

def build_model():
    base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
    base_model.trainable = False

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(4, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

    return model


Bu Python kodu, görüntü sınıflandırması için MobileNetV2 tabanlı bir derin öğrenme modeli oluşturur. Önceden eğitilmiş MobileNetV2 katmanları dondurulur ve üzerine global ortalama havuzlama, iki adet düzenlileştirilmiş (L2), batch normalization uygulanmış ve dropout katmanlarına sahip yoğun katmanlar eklenir. Son olarak, 4 sınıflı çıktı için softmax aktivasyonlu bir yoğun katman kullanılır. Model, Adam optimizasyon algoritması ve kategorik çapraz entropi kaybı ile derlenir.

In [8]:
model_orig = build_model()
history_orig = model_orig.fit(train_orig, validation_data=val_orig, epochs=10)




Epoch 1/10


20/20 [==============================] - 17s 503ms/step - loss: 2.4193 - accuracy: 0.2969 - val_loss: 1.8145 - val_accuracy: 0.4750
Epoch 2/10
20/20 [==============================] - 9s 440ms/step - loss: 2.0837 - accuracy: 0.4281 - val_loss: 1.7050 - val_accuracy: 0.5312
Epoch 3/10
20/20 [==============================] - 9s 428ms/step - loss: 1.8033 - accuracy: 0.5250 - val_loss: 1.6306 - val_accuracy: 0.5562
Epoch 4/10
20/20 [==============================] - 9s 429ms/step - loss: 1.7234 - accuracy: 0.5688 - val_loss: 1.5856 - val_accuracy: 0.6000
Epoch 5/10
20/20 [==============================] - 8s 421ms/step - loss: 1.6638 - accuracy: 0.5906 - val_loss: 1.5469 - val_accuracy: 0.6250
Epoch 6/10
20/20 [==============================] - 8s 417ms/step - loss: 1.5584 - accuracy: 0.6469 - val_loss: 1.5073 - val_accuracy: 0.6625
Epoch 7/10
20/20 [==============================] - 8s 421ms/step - loss: 1.5050 - accuracy: 0.6438 - val_loss: 1.4844 - val_accuracy: 0.6687
E

Bu Python kodu, önceden tanımlanmış olan `build_model()` fonksiyonunu çağırarak `model_orig` adında bir derin öğrenme modeli oluşturur. Ardından, bu modeli eğitmek için `fit()` metodu kullanılır. Eğitim verisi olarak `train_orig` (orijinal görüntülerden oluşturulmuş eğitim veri üreteci) ve doğrulama verisi olarak `val_orig` (orijinal görüntülerden oluşturulmuş doğrulama veri üreteci) kullanılır. Eğitim işlemi toplamda 10 epoch boyunca gerçekleştirilir. Her bir epoch, tüm eğitim verisinin bir kez model tarafından geçirilmesini ifade eder. Eğitim süresince, modelin eğitim verisi üzerindeki performansı ve doğrulama verisi üzerindeki performansı (kayıp ve doğruluk gibi metrikler) takip edilir ve bu bilgiler `history_orig` değişkenine kaydedilir. Bu değişken, eğitim sürecinin analiz edilmesi ve modelin performansının değerlendirilmesi için kullanılabilir.

In [10]:
model_proc = build_model()
history_proc = model_proc.fit(train_proc, validation_data=val_proc, epochs=15)


Epoch 1/15
160/160 [==============================] - 72s 415ms/step - loss: 2.3576 - accuracy: 0.3502 - val_loss: 1.6943 - val_accuracy: 0.5203
Epoch 2/15
160/160 [==============================] - 68s 425ms/step - loss: 2.0130 - accuracy: 0.4424 - val_loss: 1.5352 - val_accuracy: 0.6086
Epoch 3/15
160/160 [==============================] - 68s 425ms/step - loss: 1.8654 - accuracy: 0.5008 - val_loss: 1.4350 - val_accuracy: 0.6656
Epoch 4/15
160/160 [==============================] - 67s 417ms/step - loss: 1.7821 - accuracy: 0.5195 - val_loss: 1.3927 - val_accuracy: 0.6875
Epoch 5/15
160/160 [==============================] - 67s 418ms/step - loss: 1.6872 - accuracy: 0.5576 - val_loss: 1.3933 - val_accuracy: 0.6789
Epoch 6/15
160/160 [==============================] - 66s 416ms/step - loss: 1.6496 - accuracy: 0.5686 - val_loss: 1.3523 - val_accuracy: 0.6945
Epoch 7/15
160/160 [==============================] - 67s 418ms/step - loss: 1.5707 - accuracy: 0.5953 - val_loss: 1.2965 - val_ac

Bu Python kodu, önceden tanımlanmış olan `build_model()` fonksiyonunu kullanarak `model_proc` adında yeni bir derin öğrenme modeli oluşturur. Bu model, orijinal görüntülerle eğitilen `model_orig` modelinden ayrı bir örnektir. Ardından, bu yeni model, `fit()` metodu aracılığıyla eğitilir. Eğitim için, işlenmiş görüntülerden oluşturulmuş eğitim veri kümesi (`train_proc`) ve doğrulama veri kümesi (`val_proc`) kullanılır. Model, bu veri kümeleri üzerinde 15 epoch boyunca eğitilir. Her bir epoch, tüm eğitim verisinin model tarafından bir kez işlenmesi anlamına gelir. Eğitim süresince, modelin hem eğitim hem de doğrulama verisi üzerindeki performansı (örneğin, kayıp ve doğruluk değerleri) izlenir ve bu bilgiler `history_proc` değişkeninde saklanır. Bu `history_proc` nesnesi, eğitim sürecinin sonunda modelin performansını değerlendirmek ve görselleştirmek için kullanılabilir.

In [12]:
def evaluate_model(model, val_data):
    val_data.reset()
    predictions = model.predict(val_data)
    y_pred = np.argmax(predictions, axis=1)
    y_true = val_data.classes

    print(classification_report(y_true, y_pred, target_names=val_data.class_indices.keys()))
    return classification_report(y_true, y_pred, output_dict=True)

report_orig = evaluate_model(model_orig, val_orig)
report_proc = evaluate_model(model_proc, val_proc)


5/5 [==============================] - 3s 350ms/step
              precision    recall  f1-score   support

         CNV       0.76      0.88      0.81        40
         DME       0.79      0.55      0.65        40
      DRUSEN       0.59      0.60      0.59        40
      NORMAL       0.69      0.78      0.73        40

    accuracy                           0.70       160
   macro avg       0.71      0.70      0.70       160
weighted avg       0.71      0.70      0.70       160

40/40 [==============================] - 14s 305ms/step
              precision    recall  f1-score   support

         CNV       0.80      0.84      0.82       320
         DME       0.84      0.68      0.75       320
      DRUSEN       0.65      0.72      0.68       320
      NORMAL       0.74      0.76      0.75       320

    accuracy                           0.75      1280
   macro avg       0.76      0.75      0.75      1280
weighted avg       0.76      0.75      0.75      1280



In [14]:
import pandas as pd

df_orig = pd.DataFrame(report_orig).transpose()
df_proc = pd.DataFrame(report_proc).transpose()

comparison_df = pd.concat([df_orig['precision'], df_proc['precision'],
                           df_orig['recall'], df_proc['recall'],
                           df_orig['f1-score'], df_proc['f1-score']], axis=1)

comparison_df.columns = ['Orig_Precision', 'Proc_Precision', 
                         'Orig_Recall', 'Proc_Recall', 
                         'Orig_F1', 'Proc_F1']

comparison_df


,Orig_Precision,Proc_Precision,Orig_Recall,Proc_Recall,Orig_F1,Proc_F1
0,0.760870,0.795252,0.875,0.83750,0.813953,0.815830
1,0.785714,0.838462,0.550,0.68125,0.647059,0.751724
2,0.585366,0.648045,0.600,0.72500,0.592593,0.684366
3,0.688889,0.744615,0.775,0.75625,0.729412,0.750388
accuracy,0.700000,0.750000,0.700,0.75000,0.700000,0.750000
macro avg,0.705210,0.756593,0.700,0.75000,0.695754,0.750577
weighted avg,0.705210,0.756593,0.700,0.75000,0.695754,0.750577


Ödevde bizden istendiği gibi başarı artışı gözlemlenmektedir.